In [1]:
# Robert K Burns
# DSC650 Assignment 8.2
# February 2, 2020

In [2]:
#1. Stream Directory Data

#In the first part of the exercise, you will create a simple Spark streaming program that reads an input stream from a file source. The file source stream reader reads data from a directory on a file system. When a new file is added to the folder, Spark adds that file’s data to the input data stream.

#You can find the input data for this exercise in the baby-names/streaming directory. This directory contains the baby names CSV file randomized and split into 98 individual files. You will use these files to simulate incoming streaming data.

#a. Count the Number of Females

#In the first part of the exercise, you will create a Spark program that monitors an incoming directory. To simulate streaming data, you will copy CSV files from the baby-names/streaming directory into the incoming directory. Since you will be loading CSV data, you will need to define a schema before you initialize the streaming dataframe.

#From this input data stream, you will create a simple output data stream that counts the number of females and writes it to the console. Approximately every 10 seconds or so, copy a new file into the directory and report the console output. Do this for the first ten files.

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession.builder.appName('Assignment_8_2').getOrCreate()

In [4]:
dataSchema = StructType([StructField('state',StringType(),True),
                          StructField('sex',StringType(),True),
                          StructField('year',IntegerType(),True),
                          StructField('name',StringType(),True),
                          StructField('count',IntegerType(),True)])

streaming = spark.readStream.schema(dataSchema).option("maxFilesPerTrigger", 1).csv('/FileStore/tables/baby-names/streaming/', header=True)

counts = streaming.groupBy("sex").count()

In [5]:
query = counts.writeStream.queryName("counts").format("memory").outputMode("complete").start()


--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-2301029369966131> in <module> 
 ----> 1 query = counts . writeStream . queryName ( "counts" ) . format ( "memory" ) . outputMode ( "complete" ) . start ( ) 

 /databricks/spark/python/pyspark/sql/streaming.py in start (self, path, format, outputMode, partitionBy, queryName, **options) 
 1120 self . queryName ( queryName ) 
 1121 if path is None : 
 -> 1122 return self . _sq ( self . _jwrite . start ( ) ) 
 1123 else : 
 1124 return self . _sq ( self . _jwrite . start ( path ) ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 61 def deco ( * a , ** kw ) : 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 
 65 s = e . java_exception . toString ( ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 
 330 raise Py4JError(

 Py4JJavaError : An error occurred while calling o342.start.
: java.rmi.RemoteException: com.databricks.api.base.DatabricksServiceException: QUOTA_EXCEEDED: You have exceeded the maximum number of allowed files on Databricks Community Edition. To ensure free access, you are limited to 10000 files and 10 GB of storage in DBFS. Please use dbutils.fs to list and clean up files to restore service. You may have to wait a few minutes after cleaning up the files for the quota to be refreshed. (Files found: 17012); nested exception is: 
	com.databricks.api.base.DatabricksServiceException: QUOTA_EXCEEDED: You have exceeded the maximum number of allowed files on Databricks Community Edition. To ensure free access, you are limited to 10000 files and 10 GB of storage in DBFS. Please use dbutils.fs to list and clean up files to restore service. You may have to wait a few minutes after cleaning up the files for the quota to be refreshed. (Files found: 17012)
	at com.databricks.backend.daemon.data.client.DbfsClient.send0(DbfsClient.scala:123)
	at com.databricks.backend.daemon.data.client.DbfsClient.sendIgnoreDraining(DbfsClient.scala:83)
	at com.databricks.backend.daemon.data.client.DbfsOutputStream.close0(DbfsOutputStream.scala:100)
	at com.databricks.backend.daemon.data.client.DbfsOutputStream.close(DbfsOutputStream.scala:79)
	at java.io.FilterOutputStream.close(FilterOutputStream.java:159)
	at org.apache.hadoop.fs.FSDataOutputStream$PositionCache.close(FSDataOutputStream.java:72)
	at org.apache.hadoop.fs.FSDataOutputStream.close(FSDataOutputStream.java:106)
	at org.apache.hadoop.fs.FSDataOutputStream$PositionCache.close(FSDataOutputStream.java:72)
	at org.apache.hadoop.fs.FSDataOutputStream.close(FSDataOutputStream.java:106)
	at sun.nio.cs.StreamEncoder.implClose(StreamEncoder.java:320)
	at sun.nio.cs.StreamEncoder.close(StreamEncoder.java:149)
	at java.io.OutputStreamWriter.close(OutputStreamWriter.java:233)
	at com.fasterxml.jackson.core.json.WriterBasedJsonGenerator.close(WriterBasedJsonGenerator.java:883)
	at com.fasterxml.jackson.databind.ObjectMapper._configAndWriteValue(ObjectMapper.java:3561)
	at com.fasterxml.jackson.databind.ObjectMapper.writeValue(ObjectMapper.java:2909)
	at org.json4s.jackson.Serialization$.write(Serialization.scala:27)
	at org.apache.spark.sql.execution.streaming.StreamMetadata$.write(StreamMetadata.scala:80)
	at org.apache.spark.sql.execution.streaming.StreamExecution$$anonfun$2.apply(StreamExecution.scala:128)
	at org.apache.spark.sql.execution.streaming.StreamExecution$$

In [6]:
spark.sql("SELECT * FROM counts WHERE sex='F'").show()


+---+------+
sex| count|
+---+------+
 F|836266|
+---+------+

In [7]:
#2. Micro-Batching
#Repeat the last step, but use a micro-batch interval to trigger the processing every 30 seconds. Approximately every 10 seconds or so, copy a new file into the directory and report the console output. Do this for the first ten files. How did the output differ from the previous example?

In [8]:
dataSchema = StructType([StructField('state',StringType(),True),
                          StructField('sex',StringType(),True),
                          StructField('year',IntegerType(),True),
                          StructField('name',StringType(),True),
                          StructField('count',IntegerType(),True)])

micro_batch = spark.readStream.schema(dataSchema).csv('/FileStore/tables/baby-names/streaming/', header=True)
counts2 = micro_batch.groupBy("sex").count()

microbatch_writer = counts.writeStream.trigger(processingTime = '30 seconds').queryName("counts2").format("memory").outputMode("complete").start()
microbatch_writer.isActive

Out[10]: True

In [9]:
spark.sql("SELECT * FROM counts WHERE sex='F'").show()


+---+-------+
sex| count|
+---+-------+
 F|1974623|
+---+-------+

In [10]:
spark.sql("SELECT * FROM counts WHERE sex='F'").show()


+---+-------+
sex| count|
+---+-------+
 F|2074791|
+---+-------+

In [11]:
spark.sql("SELECT * FROM counts WHERE sex='F'").show()
